# Анализ факторов, влияющих на успеваемость студентов

## Подготовка к работе

In [1]:
# Импорт необходимых для работы библиотек

import pandas as pd
from scipy import stats as st

In [2]:
# Отключение предупреждений в итоговом отчёте

import warnings
warnings.filterwarnings('ignore')

In [3]:
# Выгрузка предобработанной версии датасета `jamb` 

jamb = pd.read_csv('/Users/mimikhailova/ВШЭ/jamb.csv')

In [4]:
jamb.head()

,jamb_score,study_hours_per_week,attendance_rate,teacher_quality,distance_to_school,school_type,school_location,extra_tutorials,access_to_learning_materials,parent_involvement,it_knowledge,age,gender,socioeconomic_status,parent_education_level,assignments_completed
0,192,22,78,4,12.4,Public,Urban,Yes,Yes,High,Medium,17,Male,Low,Tertiary,2
1,207,14,88,4,2.7,Public,Rural,No,Yes,High,High,15,Male,High,Unknown,1
2,182,29,87,2,9.6,Public,Rural,Yes,Yes,High,Medium,20,Female,High,Tertiary,2
3,210,29,99,2,2.6,Public,Urban,No,Yes,Medium,High,22,Female,Medium,Tertiary,1
4,199,12,98,3,8.8,Public,Urban,No,Yes,Medium,Medium,22,Female,Medium,Tertiary,1


## Проверка гипотез

### Можно ли сказать, что экзамен лучше сдают те студенты, чьи родители принимают активное участие в учебном процессе?

**Предпосылки:** 

**Сформулируем нулевую гипотезу Н0:** средний балл за экзамен JAMB у студентов с высокой вовлеченностью родителей в учебный процесс равен среднему баллу студентов с низкой вовлеченностью родителей в учебный процесс. 


**Тогда альтернативная правосторонняя гипотеза Н1 будет звучать так:** средний балл за экзамен JAMB у студентов с высокой вовлеченностью родителей в учебный процесс больше среднего балла студентов с низкой вовлеченностью родителей в учебный процесс. 

In [5]:
# Срезы по уровню вовлеченности родителей в учебный процесс

high_involvement = jamb[jamb['parent_involvement'] == 'High']
low_involvement = jamb[jamb['parent_involvement'] == 'Low']

Чтобы выбрать нужный тест для проверки гипотезы, проверим сначала данные на нормальность с помощью теста Шапиро-Уилка:

In [6]:
# Тест Шапиро-Уилка 

shapiro_high = st.shapiro(high_involvement['jamb_score'])
print(f"Для высокой вовлеченнности: statistic = {shapiro_high.statistic}, p-value = {shapiro_high.pvalue}")

shapiro_low = st.shapiro(low_involvement['jamb_score'])
print(f"Для низкой вовлеченнности:statistic = {shapiro_low.statistic}, p-value ={shapiro_low.pvalue}")

alpha = 0.05

if shapiro_high.pvalue < alpha:
    print("Данные в группе с высокой вовлеченностью родителей не распределены нормально")
else:
    print("Данные в группе с высокой вовлеченностью родителей распределены нормально")
    
if shapiro_low.pvalue < alpha:
    print("Данные в группе с низкой вовлеченностью родителей не распределены нормально")
else:
    print("Данные в группе с низкой вовлеченностью родителей распределены нормально")

Для высокой вовлеченнности: statistic = 0.9843418002128601, p-value = 1.479446565255671e-09
Для низкой вовлеченнности:statistic = 0.958748996257782, p-value =1.1650108631937777e-22
Данные в группе с высокой вовлеченностью родителей не распределены нормально
Данные в группе с низкой вовлеченностью родителей не распределены нормально


Видим, что данные групп не распределены нормально, а значит, поэтому использовать t-тест нежелательно. 

В таком случае воспользуемся **тестом Манна-Уитни** для сравнения двух независимых групп. Переформулируем гипотезы:

**Н0:** распределение баллов за экзамен JAMB у студентов с высокой вовлеченностью родителей в учебный процесс такое же, как у студентов с низкой вовлеченностью родителей в учебный процесс.


**Н1:** распределение баллов за экзамен JAMB у студентов с высокой вовлеченностью родителей в учебный процесс смещено в сторону более высоких значений по сравнению с группой низкой вовлеченностью родителей в учебный процесс.

In [7]:
# Тест Манна-Уитни для двух независимых выборок

alpha = 0.05

result_1 = st.mannwhitneyu(high_involvement['jamb_score'], low_involvement['jamb_score'], alternative='greater')

print(f"U-статистика: {result_1.statistic:.2f}, p-значение: {result_1.pvalue}")
  
if result_1.pvalue < alpha:
      print('Отвергаем нулевую гипотезу')
else:
      print('Не получилось отвергнуть нулевую гипотезу')

U-статистика: 1313114.50, p-значение: 8.841909442488723e-33
Отвергаем нулевую гипотезу


📌 **Выводы:**
* Результаты теста показали, что разница между группами статистически значима. Есть основания отвергнуть нулевую гипотезу и сделать вывод, что баллы JAMB у студентов с высокой вовлеченностью родителей в учебный процесс статистически выше, чем у студентов с низкой вовлеченностью родителей в учебный процесс.

### Верно ли, что студенты, проживающие ближе к месту обучения, сдают экзамены лучше?

**Предпосылки:** 

Медианное значение расстояния равно 10.0 км, то есть ровно половина студентов проживает на расстоянии менее 10 км от школы, а ровно половина – более. Сделаем срезы на основании этого значения:

In [8]:
distance_less = jamb[jamb['distance_to_school'] <= 10]
distance_more = jamb[jamb['distance_to_school'] > 10]

Чтобы сформулировать гипотезы и выбрать нужный тест, проверим данные на нормальность с помощью теста Шапиро-Уилка:

In [9]:
# Тест Шапиро-Уилка 

shapiro_less = st.shapiro(distance_less['jamb_score'])
print(f"Для высокой вовлеченнности: statistic = {shapiro_high.statistic}, p-value = {shapiro_high.pvalue}")

shapiro_more = st.shapiro(distance_more['jamb_score'])
print(f"Для низкой вовлеченнности:statistic = {shapiro_low.statistic}, p-value = {shapiro_low.pvalue}")

alpha = 0.05

if shapiro_high.pvalue < alpha:
    print("Данные в группе c расстоянием менее 10 км до школы распределены нормально")
else:
    print("Данные в группе c расстоянием более 10 км до школы распределены нормально")
    
if shapiro_low.pvalue < alpha:
    print("Данные в группе c расстоянием менее 10 км до школы не распределены нормально")
else:
    print("Данные в группе c расстоянием более 10 км до школы распределены нормально")

Для высокой вовлеченнности: statistic = 0.9843418002128601, p-value = 1.479446565255671e-09
Для низкой вовлеченнности:statistic = 0.958748996257782, p-value = 1.1650108631937777e-22
Данные в группе c расстоянием менее 10 км до школы распределены нормально
Данные в группе c расстоянием менее 10 км до школы не распределены нормально


Видим, что данные групп не распределены нормально, а значит, поэтому использовать t-тест нежелательно. 

В таком случае воспользуемся **тестом Манна-Уитни** для сравнения двух независимых групп. Сформулируем гипотезы:

**Нулевая гипотеза Н0:** распределения баллов JAMB у студентов, проживающих ≤10 км и >10 км от школы, одинаковы.

**Альтернативная правосторонняя гипотеза Н1**: студенты, которые живут ближе к школе (≤10 км), имеют более высокие баллы за экзамен JAMB, чем студенты, живущие дальше (>10 км).

In [10]:
# Тест Манна-Уитни для двух независимых выборок

alpha = 0.05

result_2 = st.mannwhitneyu(distance_less['jamb_score'], distance_more['jamb_score'], alternative='greater')

print(f"U-статистика: {result_2.statistic:.2f}, p-значение: {result_1.pvalue}")
  
if result_1.pvalue < alpha:
      print('Отвергаем нулевую гипотезу')
else:
      print('Не получилось отвергнуть нулевую гипотезу')

U-статистика: 3390343.50, p-значение: 8.841909442488723e-33
Отвергаем нулевую гипотезу


📌 **Выводы:**
* Результаты теста Манна-Уитни показали статистически значимое различие между баллами JAMB студентов, проживающих на расстоянии менее или равном 10 км от школы, и студентов, проживающих дальше. Р-значение значительно меньше уровня значимости, поэтому можно отвергнуть нулевую гипотезу. Баллы студентов, живущих ближе к школе, статистически выше, чем баллы студентов, живущих на большем расстоянии.